In [13]:
import os
import sys
from pathlib import Path
import logging
import numpy as np
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

In [14]:
notebook_dir = Path.cwd()
project_root = notebook_dir.parent  # If notebooks are in langechain/notebooks

# Add project root to Python path for imports
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root: {project_root}")

Project root: /app


In [15]:
from src.data_pipline.json_loader.json_loader_epo import EPOPatentLoader
from config.json_loader_config import PatentLoaderConfig
from src.embeddings.text_Chunker import TextChunker, ChunkerConfig


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

In [17]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("patent_pipeline")

In [19]:
data_dir = Path(project_root) / "data" / "parsed" / "EPO"
model_name = "sentence-transformers/all-mpnet-base-v2"
vector_db_dir = Path(project_root) / "data" / "vector_db"
vector_db_dir.mkdir(parents=True, exist_ok=True)

In [20]:
loader_config = PatentLoaderConfig(
    max_files=50,
    
    verbose=False,
    epo_dir=data_dir,
    include_abstract=True,
    include_claims=True,
    include_description=True
)
patent_loader = EPOPatentLoader(config=loader_config)
print("✅ Patent loader initialized")

✅ Patent loader initialized


In [21]:
json_files = patent_loader.load_raw_json_files()

2025-06-20 08:25:20,774 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 50 files out of 50 total files
2025-06-20 08:25:20,774 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 50 files out of 50 total files
2025-06-20 08:25:20,774 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 50 JSON files in /app/data/parsed/EPO
2025-06-20 08:25:20,774 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 50 JSON files in /app/data/parsed/EPO
2025-06-20 08:25:20,787 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 50 raw JSON files
2025-06-20 08:25:20,787 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 50 raw JSON files


In [22]:
documents  = patent_loader.extract_document_components()

2025-06-20 08:25:22,392 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 50 files out of 50 total files
2025-06-20 08:25:22,392 - src.data_pipline.json_loader.json_loader_patent - INFO - Limited to 50 files out of 50 total files
2025-06-20 08:25:22,393 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 50 JSON files in /app/data/parsed/EPO
2025-06-20 08:25:22,393 - src.data_pipline.json_loader.json_loader_patent - INFO - Found 50 JSON files in /app/data/parsed/EPO
2025-06-20 08:25:22,405 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 50 documents from 50 files
2025-06-20 08:25:22,405 - src.data_pipline.json_loader.json_loader_patent - INFO - Successfully loaded 50 documents from 50 files


In [23]:
json_files = []

In [25]:
documents[0][0]

Document(metadata={'doc_id': 'EP13899497B9W1', 'language': 'en', 'country': 'EP', 'doc_number': '3084761', 'application_number': '13899497.5', 'publication_date': '20250611', 'ipc_classes': 'G10L  19/038       20130101AFI20170426BHEP, G10L  19/07        20130101ALI20170426BHEP', 'file': 'EP13899497W1B9.xml', 'filePath': '/app/data/parsed/EPO/EPRTBJV2025000024001001/EPW1B9/EP13899497W1B9/EP13899497W1B9.json', 'title': 'AUDIO SIGNAL ENCODER', 'section': 'title'}, page_content='AUDIO SIGNAL ENCODER')

In [24]:
print(f"✅ Loaded {len(documents)} document segments from {len(json_files)} patent files")


✅ Loaded 50 document segments from 0 patent files


In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
chunker_config = ChunkerConfig(
    base_chunk_size=512,  # Adjust based on your needs
    chunk_overlap=50,
    min_chunk_size=100
)
chunker = TextChunker(tokenizer, config=chunker_config)
print("✅ TextChunker initialized")

✅ TextChunker initialized


In [27]:
print("✂️ Chunking documents...")
chunked_docs = []

for docs in tqdm(documents, desc="Chunking"):
    for doc in tqdm(docs, desc="Chunking"):
        try:
            text = doc.page_content
            metadata = doc.metadata
            
            # Use your TextChunker to split the document
            chunks = chunker.chunk_text(text, use_semantic=True)
            
            # Create LangChain Document objects from chunks
            for i, chunk_text in enumerate(chunks):
                # Copy metadata and add chunk information
                chunk_metadata = metadata.copy()
                chunk_metadata.update({
                    "chunk_index": i,
                    "total_chunks": len(chunks)
                })
                
                chunked_docs.append(Document(
                    page_content=chunk_text,
                    metadata=chunk_metadata
                ))
        except Exception as e:
            logger.error(f"Error chunking document: {str(e)}")

✂️ Chunking documents...


Chunking:   0%|          | 0/50 [00:00<?, ?it/s]

Chunking:   0%|          | 0/180 [00:00<?, ?it/s]

2025-06-20 08:26:30,094 - src.embeddings.text_Chunker - INFO - Input text has 5 tokens
2025-06-20 08:26:30,095 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:30,095 - src.embeddings.text_Chunker - INFO - Input text has 33 tokens
2025-06-20 08:26:30,096 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:30,096 - src.embeddings.text_Chunker - INFO - Input text has 33 tokens
2025-06-20 08:26:30,096 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:30,097 - src.embeddings.text_Chunker - INFO - Input text has 116 tokens
2025-06-20 08:26:30,097 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:30,098 - src.embeddings.text_Chunker - INFO - Input text has 37 tokens
2025-06-20 08:26:30,099 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:30,099 - src.embeddings.text_Chunker - INFO - Input text has 88 tokens
2025-06-20 08:26:30,099 - s

Chunking:   0%|          | 0/16 [00:00<?, ?it/s]

...

2025-06-20 08:26:31,062 - src.embeddings.text_Chunker - INFO - Input text has 18 tokens
2025-06-20 08:26:31,063 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,064 - src.embeddings.text_Chunker - INFO - Input text has 495 tokens
2025-06-20 08:26:31,064 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,064 - src.embeddings.text_Chunker - INFO - Input text has 10 tokens
2025-06-20 08:26:31,065 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,065 - src.embeddings.text_Chunker - INFO - Input text has 10 tokens
2025-06-20 08:26:31,065 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,065 - src.embeddings.text_Chunker - INFO - Input text has 10 tokens
2025-06-20 08:26:31,065 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,066 - src.embeddings.text_Chunker - INFO - Input text has 8 tokens
2025-06-20 08:26:31,066 - s

Chunking:   0%|          | 0/62 [00:00<?, ?it/s]

2025-06-20 08:26:31,069 - src.embeddings.text_Chunker - INFO - Input text has 4 tokens
2025-06-20 08:26:31,069 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,070 - src.embeddings.text_Chunker - INFO - Input text has 40 tokens
2025-06-20 08:26:31,070 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,070 - src.embeddings.text_Chunker - INFO - Input text has 96 tokens
2025-06-20 08:26:31,071 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,071 - src.embeddings.text_Chunker - INFO - Input text has 99 tokens
2025-06-20 08:26:31,071 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,072 - src.embeddings.text_Chunker - INFO - Input text has 57 tokens
2025-06-20 08:26:31,072 - src.embeddings.text_Chunker - INFO - Text fits in single chunk
2025-06-20 08:26:31,072 - src.embeddings.text_Chunker - INFO - Input text has 65 tokens
2025-06-20 08:26:31,072 - sr

Chunking:   0%|          | 0/77 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
documents = []
chunked_docs[0]

Document(metadata={'doc_id': 'EP13899497B9W1', 'language': 'en', 'country': 'EP', 'doc_number': '3084761', 'application_number': '13899497.5', 'publication_date': '20250611', 'ipc_classes': 'G10L  19/038       20130101AFI20170426BHEP, G10L  19/07        20130101ALI20170426BHEP', 'file': 'EP13899497W1B9.xml', 'filePath': '/app/data/parsed/EPO/EPRTBJV2025000024001001/EPW1B9/EP13899497W1B9/EP13899497W1B9.json', 'title': 'AUDIO SIGNAL ENCODER', 'section': 'title', 'chunk_index': 0, 'total_chunks': 1}, page_content='AUDIO SIGNAL ENCODER')

In [29]:
print(f"✅ Created {len(chunked_docs)} chunks from {len(documents)} document segments")


✅ Created 1409 chunks from 0 document segments


In [30]:

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"batch_size": 32, "normalize_embeddings": True}
)
print("✅ LangChain embeddings initialized")


/tmp/ipykernel_4798/3069164953.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-06-20 08:30:07,970 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


✅ LangChain embeddings initialized


In [4]:
# 7. Create vector store with chunked documents - connect to Docker ChromaDB
print("💾 Creating vector store using dockerized ChromaDB...")

# Import ChromaDB client directly (needed for Docker connection)
import chromadb
from chromadb.config import Settings

# When running inside Docker
chroma_client = chromadb.HttpClient(
    host="vector_db",  # Docker service name
    port=8000,         # Internal container port
    settings=Settings(anonymized_telemetry=False)
)

💾 Creating vector store using dockerized ChromaDB...


In [33]:
collection_name = "patents"

vectorstore = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=embeddings
)

2025-06-20 08:30:50,248 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections "HTTP/1.1 200 OK"


In [34]:
# Add documents to the vectorstore (if we have any)
if chunked_docs:
    # Add in smaller batches to avoid memory issues
    batch_size = 100
    for i in tqdm(range(0, len(chunked_docs), batch_size), desc="Adding to vector store"):
        batch = chunked_docs[i:i+batch_size]
        vectorstore.add_documents(documents=batch)
    
    print(f"✅ Added {len(chunked_docs)} document chunks to ChromaDB collection '{collection_name}'")
else:
    print("⚠️ No document chunks to add to the vector store!")

Adding to vector store:   0%|          | 0/15 [00:00<?, ?it/s]

2025-06-20 08:31:05,094 - httpx - INFO - HTTP Request: GET http://vector_db:8000/api/v2/pre-flight-checks "HTTP/1.1 200 OK"
2025-06-20 08:31:05,250 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/34b877df-69c0-4489-8a41-83afcc841670/upsert "HTTP/1.1 200 OK"
2025-06-20 08:31:14,596 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/34b877df-69c0-4489-8a41-83afcc841670/upsert "HTTP/1.1 200 OK"
2025-06-20 08:31:25,186 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/34b877df-69c0-4489-8a41-83afcc841670/upsert "HTTP/1.1 200 OK"
2025-06-20 08:31:35,209 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/34b877df-69c0-4489-8a41-83afcc841670/upsert "HTTP/1.1 200 OK"
2025-06-20 08:31:46,008 

✅ Added 1409 document chunks to ChromaDB collection 'patents'


In [38]:
# 8. Test retrieval (only if we have documents)
if chunked_docs:
    print("\n🔍 Testing vector store with sample queries...")

    test_queries = [
        "Audio signal encoding methods",
        # "Image compression techniques",
        # "Wireless communication protocols"
    ]

    for query in test_queries:
        print(f"\nQuery: '{query}'")
        # Get top 3 results
        results = vectorstore.similarity_search_with_score(query, k=3)
        
        print(f"Top {len(results)} results:")
        for i, (doc, score) in enumerate(results):
            print(f"\nResult {i+1} (similarity: {score:.4f})")
            print(f"Title: {doc.metadata.get('title', 'N/A')}")
            print(f"Section: {doc.metadata.get('section', 'N/A')}")
            print(f"Patent ID: {doc.metadata.get('doc_id', 'N/A')}")
            print(f"file Path: {doc.metadata.get('filePath', 'N/A')}")
            print(f"Content preview: {doc.page_content[:150]}...")
else:
    print("\n⚠️ No documents in vector store to search!")

2025-06-20 08:36:15,588 - httpx - INFO - HTTP Request: POST http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/34b877df-69c0-4489-8a41-83afcc841670/query "HTTP/1.1 200 OK"



🔍 Testing vector store with sample queries...

Query: 'Audio signal encoding methods'
Top 3 results:

Result 1 (similarity: 0.4964)
Title: AUDIO SIGNAL ENCODER
Section: Description of Some Embodiments of the Application
Patent ID: EP13899497B9W1
file Path: /app/data/parsed/EPO/EPRTBJV2025000024001001/EPW1B9/EP13899497W1B9/EP13899497W1B9.json
Content preview: Description of Some Embodiments of the Application
The concept for the embodiments as described herein is to determine and apply encoding to audio sig...

Result 2 (similarity: 0.5347)
Title: AUDIO SIGNAL ENCODER
Section: Background
Patent ID: EP13899497B9W1
file Path: /app/data/parsed/EPO/EPRTBJV2025000024001001/EPW1B9/EP13899497W1B9/EP13899497W1B9.json
Content preview: Background
Audio signals, like speech or music, are encoded for example to enable efficient transmission or storage of the audio signals. Examples of ...

Result 3 (similarity: 0.5472)
Title: AUDIO SIGNAL ENCODER
Section: Summary
Patent ID: EP13899497B9W1
file Pat

In [32]:
# vectorstore.delete_collection()

2025-06-20 08:30:44,712 - httpx - INFO - HTTP Request: DELETE http://vector_db:8000/api/v2/tenants/default_tenant/databases/default_database/collections/patents "HTTP/1.1 200 OK"
